In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra

import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
import seaborn as sns

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv("../input/train.csv",names=['id','spacegroup','numatoms','peral','perga','perin','a','b','c','alpha','beta','gamma','E0','Eg'],skiprows=1)
df_topredict = pd.read_csv("../input/test.csv",names=['id','spacegroup','numatoms','peral','perga','perin','a','b','c','alpha','beta','gamma'],skiprows=1)

Lets go ahead and see what the data to train on and the data to predict on looks like

In [ ]:
df_train.head()

In [ ]:

df_topredict.head()

In [ ]:
df_cpy = df_train.copy()
df_topredictcpy = df_topredict.copy()


In [ ]:
df_cpy.info()

In [ ]:
df_cpy.isnull().sum()

So we have no null values in the dataset.Here i am rounding of the angles. It shouldn't matter much as far the prediction algorithm goes. The reason for the rounding of the angles is that crystal structures usually have well defined angles (90, 90, 90 for cubic; 90,90,120 for hexagonal and so on).

So, lets go ahead and see how the values the lattice angles take.

In [ ]:
f,ax = plt.subplots(1,3,figsize=(10,3))
ax[0].hist(df_cpy['alpha'].unique())
ax[0].set_title("Histogram plot of alpha")
ax[0].set_xlabel("alpha")
ax[1].hist(df_cpy['beta'].unique())
ax[1].set_title("Histogram plot of beta")
ax[1].set_xlabel("beta")
ax[2].hist(df_cpy['gamma'].unique())
ax[2].set_title("Histogram plot of gamma")
ax[2].set_xlabel("gamma")

In [ ]:
df_cpy['alpha'] = np.round(df_cpy['alpha'])
df_cpy['beta'] = np.round(df_cpy['beta'])
df_cpy['gamma'] = np.round(df_cpy['gamma'])
df_topredictcpy['alpha'] = np.round(df_topredictcpy['alpha'])
df_topredictcpy['beta'] = np.round(df_topredictcpy['beta'])
df_topredictcpy['gamma'] = np.round(df_topredictcpy['gamma'])

So we se that the lattice angles alpha and beta always take values near 90 with some variation ~10 degrees, whereas the lattice angle gamma takes on either 90 or 120 or 30 degrees. We will look into more analysis of the lattice angles/ vectors with respect to individual spacegroups a bit later

In [ ]:
df_cpy.columns

In [ ]:
cols = df_cpy[['peral','perga','perin','a','b','c','alpha','beta','gamma','E0','Eg']]

**Plotting the features with respect to other features**

Since we have 9 features (lattice vectors, lattice angles, composition) and the formation enthalpy and band gap, the plots of the features look quite cluttered. For the time being let us just look at a bird's eye view of the these features and then focus more on the features which show more correlation later

In [ ]:

sns.pairplot(cols, diag_kind="kde",size=3,aspect=1)


In [ ]:
f,ax = plt.subplots(5,3,figsize=(20,20))
ax[0,0].plot(df_cpy['alpha'],df_cpy['beta'],'ro')
ax[0,0].set_xlabel("alpha",fontweight='bold',fontsize=16)
ax[0,0].set_ylabel("beta",fontweight='bold',fontsize=16)
ax[0,1].plot(df_cpy['alpha'],df_cpy['gamma'],'bo')
ax[0,1].set_xlabel("alpha",fontweight='bold',fontsize=16)
ax[0,1].set_ylabel("gamma",fontweight='bold',fontsize=16)
ax[0,2].plot(df_cpy['beta'],df_cpy['gamma'],'go')
ax[0,2].set_xlabel("beta",fontweight='bold',fontsize=16)
ax[0,2].set_ylabel("gamma",fontweight='bold',fontsize=16)
ax[1,0].plot(df_cpy['alpha'],df_cpy['a'],'ro')
ax[1,0].set_xlabel("alpha",fontweight='bold',fontsize=16)
ax[1,0].set_ylabel("a",fontweight='bold',fontsize=16)
ax[1,1].plot(df_cpy['alpha'],df_cpy['b'],'bo')
ax[1,1].set_xlabel("alpha",fontweight='bold',fontsize=16)
ax[1,1].set_ylabel("b",fontweight='bold',fontsize=16)
ax[1,2].plot(df_cpy['alpha'],df_cpy['c'],'go')
ax[1,2].set_xlabel("alpha",fontweight='bold',fontsize=16)
ax[1,2].set_ylabel("c",fontweight='bold',fontsize=16)
ax[2,0].plot(df_cpy['beta'],df_cpy['a'],'ro')
ax[2,0].set_xlabel("beta",fontweight='bold',fontsize=16)
ax[2,0].set_ylabel("a",fontweight='bold',fontsize=16)
ax[2,1].plot(df_cpy['beta'],df_cpy['b'],'bo')
ax[2,1].set_xlabel("beta",fontweight='bold',fontsize=16)
ax[2,1].set_ylabel("b",fontweight='bold',fontsize=16)
ax[2,2].plot(df_cpy['beta'],df_cpy['c'],'go')
ax[2,2].set_xlabel("beta",fontweight='bold',fontsize=16)
ax[2,2].set_ylabel("c",fontweight='bold',fontsize=16)
ax[3,0].plot(df_cpy['gamma'],df_cpy['a'],'ro')
ax[3,0].set_xlabel("gamma",fontweight='bold',fontsize=16)
ax[3,0].set_ylabel("a",fontweight='bold',fontsize=16)
ax[3,1].plot(df_cpy['gamma'],df_cpy['b'],'bo')
ax[3,1].set_xlabel("gamma",fontweight='bold',fontsize=16)
ax[3,1].set_ylabel("b",fontweight='bold')
ax[3,2].plot(df_cpy['gamma'],df_cpy['c'],'go')
ax[3,2].set_xlabel("gamma",fontweight='bold',fontsize=16)
ax[3,2].set_ylabel("c",fontweight='bold',fontsize=16)
ax[4,0].plot(df_cpy['a'],df_cpy['b'],'ro')
ax[4,0].set_xlabel("a",fontweight='bold',fontsize=16)
ax[4,0].set_ylabel("b",fontweight='bold',fontsize=16)
ax[4,1].plot(df_cpy['b'],df_cpy['c'],'bo')
ax[4,1].set_xlabel("b",fontweight='bold',fontsize=16)
ax[4,1].set_ylabel("c",fontweight='bold',fontsize=16)
ax[4,2].plot(df_cpy['a'],df_cpy['c'],'go')
ax[4,2].set_xlabel("a",fontweight='bold',fontsize=16)
ax[4,2].set_ylabel("c",fontweight='bold',fontsize=16)

So we see that there is a strong positive correlation  between alpha and beta, 

**Correlation Matrix**

The correlation matrix is similar to the pair plotting of the features we saw above but is more quantitative and helps us understand more which the plots dont give away readily. Now lets go ahead and see how the features are correlated and see if they have any similarities.

In [ ]:
cols = df_cpy[['spacegroup','alpha','beta','gamma','peral','perga','perin','a','b','c']]
corrmat = cols.corr(method='spearman')
plt.subplots(figsize=(15,12))
sns.heatmap(corrmat, vmax=1., square=True,annot=True,robust=True)

I focus only on the pairs whcih yield high values of correlation (ie >0.3) because a lot of the cases below this value might purley be an artifact of the dataset. Let us start with the lattice angles. 

Alpha has a strong positive correlation with beta, a strong negative correlation with gamma.

Beta has a strong positive correlation with alpha, a strong negative correlation with gamma, a very strong positive correlation with a, and strong negative correlations with b,c.

Gamma has strong negative correlations with alpha, beta, b, strong positive correlation with c and a very strong negative correlation with a.

peral (aluminium composition) has strong negative correlations with per in (indium composition) and per ga (gallium composition). 

perga (gallium composition) has strong negative correlations with per al  and per in. 

perin (indium composition) has strong negative correlations with per al  and per ga and a strong positive b 


.  In principle, that is not the case expected in crystal systems as the lattice angles are fully independent to vary with respect to each other. So this correlation might strongly be influenced by the particular dataset we are looking at.  Lets plot these angles wrt to each other and see how they vary.


Now, lets look at the different spacegroups and see what spacegroups the structures are taken from. 

In [ ]:
sorted(df_cpy['spacegroup'].unique())


In [ ]:
sorted(df_topredictcpy['spacegroup'].unique())

In [ ]:
df_cpy['spacegroup'].value_counts(normalize=True).plot.bar()
plt.title("Distribution of spacegroups in the sample set")
plt.xlabel("Spacegroups")
plt.ylabel("% of sample set")

In [ ]:
df_topredictcpy['spacegroup'].value_counts(normalize=True).plot.bar()
plt.title("Distribution of spacegroups in the sample set")
plt.xlabel("Spacegroups")
plt.ylabel("% of sample set")

So, we see that the samples used for training and samples to be predicted are from the same spacegroups. The distribution of the samples in the different spacegroups in both the training set and the set to be predicted takes a similar distribution. So, we know that the sample taken for training is a good statistical sample set.  Now lets look how the inidividual compositions are stacked up wrt to the different spacegroups. 


In [ ]:
sns.barplot(df_cpy['spacegroup'],df_cpy['perin'])

In [ ]:
sns.barplot(df_cpy['spacegroup'],df_cpy['perga'])

In [ ]:
sns.barplot(df_cpy['spacegroup'],df_cpy['peral'])

So we see that the compositions of aluminium, gallium and indium pretty much have similar ranges of compositons in each spacegroup. 

Now let us look at how bandgaps and formation enthalpies vary with elemental compositions

In [ ]:
f,ax = plt.subplots(1,3,figsize=(10,3))
ax[0].plot(df_cpy['perin'],df_cpy['E0'],'ro')
ax[0].set_xlabel("% of Indium")
ax[0].set_ylabel("Formation enthalpy  (eV)")
ax[1].plot(df_cpy['perga'],df_cpy['E0'],'bo')
ax[1].set_xlabel("% of Gallium")
ax[2].plot(df_cpy['peral'],df_cpy['E0'],'go')
ax[2].set_xlabel("% of Aluminium")

We see that aluminium and indium compositions dont matter too much for the formation enthalpy, but there is a systematic decrease in the formation enthalpy with increasing gallium composition

In [ ]:
f,ax = plt.subplots(1,3,figsize=(10,3))
ax[0].plot(df_cpy['perin'],df_cpy['Eg'],'ro')
ax[0].set_xlabel("% of Indium")
ax[0].set_ylabel("Band Gap  (eV)")
ax[1].plot(df_cpy['perga'],df_cpy['Eg'],'bo')
ax[1].set_xlabel("% of Gallium")
ax[2].plot(df_cpy['peral'],df_cpy['Eg'],'go')
ax[2].set_xlabel("% of Aluminium")

Unlike formation enthalpies, we see some clear trends in band gap as a function of compositions. In particular, increasing aluminium and decreasing indium leads to increasing band gap values. Increase in the gallium composition however attains a range of values (depending on the aluminium and indium compositions) and tapers out to ~ 2 eV for 100 % Gallium. So ideally for high band gap values, we would want low compositions of indium, medium compositions of gallium and high compositions of Aluminium. However, high compositions of gallium lead to low formation enthalpy values. Thus we would want compounds with high compositions of gallium and aluminium


Now,let's look at the geometric characteristics of each spacegroup based on the dataset we have

In [ ]:
crystal_vectors = ['a','b','c','alpha','beta','gamma']

In [ ]:
crystal_vectors_withbandandenergy = ['a','b','c','alpha','beta','gamma','Eg','E0','perin','peral','perga']

**Space group 12**

This is a monoclinic structure with the lattice vectors not being equal, two lattice angles are 90 and the other lattice angle can take any value

In [ ]:
#plt.subplots(figsize=(15,12))
df_cpy[df_cpy['spacegroup']==12].hist(figsize=(12,12),column=crystal_vectors_withbandandenergy)


In [ ]:
df_cpy[df_cpy['spacegroup']==12]['alpha'].unique()

So in space group 12, we see that gamma and alpha takes only one value (90), while beta takes (104,105 and 106) (ie. not 90). a,b,c are not equal, while E0 takes values from 0 - 0.4 while Eg takes on values from 0-4 centered around 2 eV. The compositions of aluminium, gallium and indium take compositions from 0-1. So there is no particular preferece for the compositions to be in particular spacegroups

In [ ]:
#plt.subplots(figsize=(15,12))
df_cpy[df_cpy['spacegroup']==33].hist(figsize=(12,12),column=crystal_vectors_withbandandenergy)


In [ ]:
#plt.subplots(figsize=(15,12))
df_cpy[df_cpy['spacegroup']==167].hist(figsize=(12,12),column=crystal_vectors_withbandandenergy)


In [ ]:
#plt.subplots(figsize=(15,12))
df_cpy[df_cpy['spacegroup']==194].hist(figsize=(12,12),column=crystal_vectors_withbandandenergy)


In [ ]:
#plt.subplots(figsize=(15,12))
df_cpy[df_cpy['spacegroup']==206].hist(figsize=(12,12),column=crystal_vectors_withbandandenergy)


In [ ]:
#plt.subplots(figsize=(15,12))
df_cpy[df_cpy['spacegroup']==227].hist(figsize=(12,12),column=crystal_vectors_withbandandenergy)


In [ ]:
df_cpy.groupby(['spacegroup'])['Eg'].describe()

In [ ]:
plt.hist(df_cpy['numatoms'])
plt.hist(df_topredictcpy['numatoms'])

So we see that the num of atoms in the training set and testing set take on same values. We can thus safely ignore the total num of atoms, because the formation energy and band gap doesnt vary with respect to varying number of total atoms as long as the composition and crystal structure remains constants. For eg, band gap of Al2O3 in space group 194 with 2 atoms of Al and 3 atoms of O would be the same as 4 atoms of Al and 6 atoms of O and so on. 

In [ ]:
df_cpy.head()

In [ ]:
df_cpy = df_cpy.drop(['numatoms'],axis=1)
df_topredictcpy = df_topredictcpy.drop(['numatoms'],axis=1)

In [ ]:
df_cpy = df_cpy.drop(['id'],axis=1)
df_topredictcpy = df_topredictcpy.drop(['id'],axis=1)

In [ ]:
df_cpy.head()

In [ ]:
df_topredictcpy.head()

**Structure mapping **

The different values of spacegroup variable (12,33,167,194,206,227) are categorical values. So we have to convert them into one-hot encoding format. 

In [ ]:
spacegroup_mapping = {12:1,33:2,167:3, 194:4, 206:5,227:6}

In [ ]:
df_cpy['structure_mapped']=df_cpy['spacegroup'].map(spacegroup_mapping)
df_topredictcpy['structure_mapped']=df_topredictcpy['spacegroup'].map(spacegroup_mapping)

In [ ]:
df_cpy = df_cpy.drop(['spacegroup'],axis=1)
df_topredictcpy = df_topredictcpy.drop(['spacegroup'],axis=1)

In [ ]:
one_hot = pd.get_dummies(df_cpy['structure_mapped'],columns=['sg12','sg33','sg167','sg194','sg206','sg227'])
one_hot_topredict = pd.get_dummies(df_topredictcpy['structure_mapped'],columns=['sg12','sg33','sg167','sg194','sg206','sg227'])

In [ ]:
df_cpy = df_cpy.join(one_hot)
df_topredictcpy = df_topredictcpy.join(one_hot_topredict)

In [ ]:
df_cpy.rename(columns={1: 'sg12', 2: 'sg33',3: 'sg167',4: 'sg194',5: 'sg206',6: 'sg227'},inplace=True)
df_topredictcpy.rename(columns={1: 'sg12', 2: 'sg33',3: 'sg167',4: 'sg194',5: 'sg206',6: 'sg227'},inplace=True)

In [ ]:
df_cpy = df_cpy.drop(['structure_mapped'],axis=1)
df_topredictcpy = df_topredictcpy.drop(['structure_mapped'],axis=1)

In [ ]:
df_cpy.head()

In [ ]:
df_topredictcpy.head()

**Feature Scaling**

Now we have manipulated the data and converted all inputs into the parameters we want to input in the ML model. However, the dataset remains a bit skewed wrt to the range of values the features take. For eg. a takes a max value of ~25, and gamma takes on 120 while the elemental compositions take on a maximum value of 1. This leads to a problem while the algorithms try to find the minimum using gradient descent. So we feature scale the values so that all features take on values from 0-1. But since features a, b,c are of the same type  and alpha,beta and gamma are of the same type, we need to maintain the relative ratios (ie a:b, alpha:beta etc). So, we are going to feature scale the lattice vectors with  a same scale and angles with a same scale.

In [ ]:
length_max = max(max(df_cpy['a']),max(df_cpy['b']),max(df_cpy['c']))
angle_max=max(max(df_cpy['alpha']),max(df_cpy['beta']),max(df_cpy['gamma']))

In [ ]:
angle_max

In [ ]:
df_cpy['alpha_reduced'] = df_cpy['alpha']/(angle_max)
df_cpy['beta_reduced'] = df_cpy['beta']/(angle_max)
df_cpy['gamma_reduced'] = df_cpy['gamma']/(angle_max)
df_topredictcpy['alpha_reduced'] = df_topredictcpy['alpha']/(angle_max)
df_topredictcpy['beta_reduced'] = df_topredictcpy['beta']/(angle_max)
df_topredictcpy['gamma_reduced'] = df_topredictcpy['gamma']/(angle_max)
df_cpy['a_reduced'] = df_cpy['a']/(length_max)
df_cpy['b_reduced'] = df_cpy['b']/(length_max)
df_cpy['c_reduced'] = df_cpy['c']/(length_max)
df_topredictcpy['a_reduced'] = df_topredictcpy['a']/(length_max)
df_topredictcpy['b_reduced'] = df_topredictcpy['b']/(length_max)
df_topredictcpy['c_reduced'] = df_topredictcpy['c']/(length_max)                                                                           

In [ ]:
df_cpy.head()

In [ ]:
df_cpy = df_cpy.drop(['alpha','beta','gamma'],axis=1)
df_topredictcpy = df_topredictcpy.drop(['alpha','beta','gamma'],axis=1)

In [ ]:
df_cpy = df_cpy.drop(['a','b','c'],axis=1)
df_topredictcpy = df_topredictcpy.drop(['a','b','c'],axis=1)

Now all our features have values in the range 0 - 1. Now lets go ahead and look at the distributions of the features with the reduced dimensions for a random spacegroup.

In [ ]:
crystal_vectors_withbandandenergy_reduced = ['a_reduced','b_reduced','c_reduced','alpha_reduced','beta_reduced','gamma_reduced','Eg','E0','perin','peral','perga']

In [ ]:
#plt.subplots(figsize=(15,12))
df_cpy[df_cpy['sg227']==1].hist(figsize=(12,12),column=crystal_vectors_withbandandenergy_reduced)


In [ ]:
df_cpy.head()

In [ ]:
df_topredictcpy.head()

In [ ]:
band = df_cpy['Eg']
energy = df_cpy['E0']

In [ ]:
df_cpy = df_cpy.drop(['Eg','E0'],axis=1)

In [ ]:
df_cpy.head()

**Running the ML algorithms:**
Now we have manipulated the data to feed into the algorithm. So we are going to split the dataset into training and testing sets. Since we have 2 target variables: formation enthalpy and band gap, we have to run algorithms for each target variable

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neural_network import  MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

X_train,X_test,Y_train,Y_test = train_test_split(df_cpy,band,test_size=0.2,random_state = 42)

In [ ]:
print (Y_test.shape)

In [ ]:
sns.distplot(Y_train)

In [ ]:
sns.distplot(Y_test)

We see that the training and testing target variable (band gap) have similar distributions. So the training and testing sets are good statistical samples of the overall sample

In [ ]:
#Y_test = np.array(Y_test)

In [ ]:
print (X_train.shape,X_test.shape)

In [ ]:
clf_tree_band = DecisionTreeRegressor(random_state = 42, max_depth=15,min_samples_split=2)
#clf_gauss = GaussianNB()
clf_kernal_band = KernelRidge(alpha=1)
clf_linear_band = LinearRegression()
clf_svr_band = SVR()
clf_neural_band = MLPRegressor(hidden_layer_sizes=10, solver="sgd",early_stopping=True)
clf_neigh_band = KNeighborsRegressor(n_neighbors=10)
clf_random_band = RandomForestRegressor(n_estimators=200)

In [ ]:
clf_tree_band.fit(X_train,Y_train)
#clf_gauss.fit(X_train,Y_train)
clf_kernal_band.fit(X_train,Y_train)
clf_linear_band.fit(X_train,Y_train)
clf_svr_band.fit(X_train,Y_train)
clf_neural_band.fit(X_train,Y_train)
clf_neigh_band.fit(X_train,Y_train)
clf_random_band.fit(X_train,Y_train)

In [ ]:
pred_tree_band = clf_tree_band.predict(X_test)
pred_kernal_band = clf_kernal_band.predict(X_test)
pred_linear_band = clf_linear_band.predict(X_test)
pred_svr_band = clf_svr_band.predict(X_test)
pred_neural_band = clf_neural_band.predict(X_test)
pred_neigh_band = clf_neigh_band.predict(X_test)
pred_random_band = clf_random_band.predict(X_test)

In [ ]:
#pred=  np.array(pred)
from sklearn.metrics import  mean_squared_error

In [ ]:
#clf_tree.score(pred,Y_test)
print(1- mean_squared_error(Y_test,pred_tree_band))
print(1- mean_squared_error(Y_test,pred_kernal_band))
print(1- mean_squared_error(Y_test,pred_linear_band))
print(1- mean_squared_error(Y_test,pred_svr_band))
print(1- mean_squared_error(Y_test,pred_neural_band))
print (1- mean_squared_error(Y_test,pred_neigh_band))
print (1- mean_squared_error(Y_test,pred_random_band))

We see that random forest algorithms gives us the best results, although the accuracy could have been better. One possible way to imporve accuracy can be to increase the number of features and number of datapoints

In [ ]:
Y_test = np.array(Y_test)
from sklearn.metrics import mean_absolute_error

In [ ]:
err_array=[];
for i in range(len(pred_random_band)):
    err = abs(pred_random_band[i]-Y_test[i])
    err_array.append(err)
max_error = max(err_array)   
print (np.mean(err_array))
print (max_error)  

    

In [ ]:
sns.distplot(err_array,bins=100)
plt.title("Distribution plot of the error in prediction for band gap")
plt.xlabel ("Error in prediction (eV)")

In [ ]:
 plt.subplots(figsize=(25,7))
sns.barplot(df_cpy.columns,clf_random_band.feature_importances_)

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.neural_network import  MLPRegressor
from sklearn.ensemble import RandomForestRegressor

X_train_enthalpy,X_test_enthalpy,Y_train_enthalpy,Y_test_enthalpy = train_test_split(df_cpy,energy,test_size=0.2,random_state = 42)

In [ ]:
clf_tree_enthalpy = DecisionTreeRegressor(random_state = 42, max_depth=10,min_samples_split=2)
#clf_gauss = GaussianNB()
clf_kernal_enthalpy = KernelRidge(alpha=1)
clf_linear_enthalpy = LinearRegression()
clf_svr_enthalpy = SVR()
clf_neural_enthalpy = MLPRegressor(hidden_layer_sizes=10, solver="sgd",early_stopping=True)
clf_random_enthalpy = RandomForestRegressor(n_estimators=100)

In [ ]:
clf_tree_enthalpy.fit(X_train_enthalpy,Y_train_enthalpy)
#clf_gauss.fit(X_train,Y_train)
clf_kernal_enthalpy.fit(X_train_enthalpy,Y_train_enthalpy)
clf_linear_enthalpy.fit(X_train_enthalpy,Y_train_enthalpy)
clf_svr_enthalpy.fit(X_train_enthalpy,Y_train_enthalpy)
clf_neural_enthalpy.fit(X_train_enthalpy,Y_train_enthalpy)
clf_random_enthalpy.fit(X_train_enthalpy,Y_train_enthalpy)

In [ ]:
pred_tree_enthalpy = clf_tree_enthalpy.predict(X_test_enthalpy)
pred_kernal_enthalpy = clf_kernal_enthalpy.predict(X_test_enthalpy)
pred_linear_enthalpy = clf_linear_enthalpy.predict(X_test_enthalpy)
pred_svr_enthalpy = clf_svr_enthalpy.predict(X_test_enthalpy)
pred_neural_enthalpy = clf_neural_enthalpy.predict(X_test_enthalpy)
pred_random_enthalpy = clf_random_enthalpy.predict(X_test_enthalpy)

In [ ]:
print(1- mean_squared_error(Y_test_enthalpy,pred_tree_enthalpy))
print(1- mean_squared_error(Y_test_enthalpy,pred_kernal_enthalpy))
print(1- mean_squared_error(Y_test_enthalpy,pred_linear_enthalpy))
print(1- mean_squared_error(Y_test_enthalpy,pred_svr_enthalpy))
#print(1- mean_squared_error(Y_test_enthalpy,pred_neigh_enthalpy))
print(1- mean_squared_error(Y_test_enthalpy,pred_neural_enthalpy))
print(1- mean_squared_error(Y_test_enthalpy,pred_random_enthalpy))

In [ ]:
Y_test_enthalpy= np.array(Y_test_enthalpy)

In [ ]:
err_array=[];
for i in range(len(pred_random_enthalpy)):
    err = abs(pred_random_enthalpy[i]-Y_test_enthalpy[i])
    err_array.append(err)
max_error = max(err_array)   
print (np.mean(err_array))
print (max_error)  

In [ ]:
sns.distplot(err_array,bins=100)
plt.title("Distribution plot of the error in prediction for enthalpy")
plt.xlabel ("Error in prediction (eV)")

In [ ]:
 plt.subplots(figsize=(25,7))
sns.barplot(df_cpy.columns,clf_random_enthalpy.feature_importances_)

So, based on the above fitting we see that indium compositions strongly affects both the formation enthalpies and band gap values (band gap more than formation enthalpies). Similarly, gallium and aluminium compositions affect the formation enthalpies but not so much for the band gap. However, what is surprising is that the lattice parameter "c" contributes very strongly to the formation enthalpy. This is likely a pure artifact of the dataset.

In [ ]:
f,ax=plt.subplots(1,2,figsize=(12,4))
#ax[0].plot(df_cpy['perin'],df_cpy['E0'],'ro')
ax[0].plot(Y_test,pred_random_band,'ro')
#plt.plot(Y_test,Y_test,'k--')
ax[0].errorbar(Y_test,Y_test,xerr=0,yerr = 0.15)
ax[0].set_xlabel("Testing set for band gap (eV)")
ax[0].set_ylabel("Predicted values for the test set for band gap (eV)")
ax[1].plot(Y_test_enthalpy,pred_random_enthalpy,'ro')
#plt.plot(Y_test,Y_test,'k--')
ax[1].errorbar(Y_test_enthalpy,Y_test_enthalpy,xerr=0,yerr = 0.03)
ax[1].set_xlabel("Testing set for formation enthalpy (eV)")
ax[1].set_ylabel("Predicted values for the test set for formation enthalpy (eV)")

In [ ]:
band_to_predict= clf_random_band.predict(df_topredictcpy)

In [ ]:
enthalpy_to_predict = clf_random_enthalpy.predict(df_topredictcpy)

In [ ]:
sns.distplot(band_to_predict,kde=False)
plt.xlabel("Band gap for the predicted set")

In [ ]:
sns.distplot(enthalpy_to_predict,kde=False)
plt.xlabel("Formation enthalpies for the predicted set")